# Part 4: Get Untappd Check-In Venue Time Zones

Get the timezone for each venue's lat-long from the Google Maps timezone API: https://developers.google.com/maps/documentation/timezone/intro

Then convert each check-in's datetime to the timezone of the venue at which it was checked in.

In [1]:
import time, requests, pytz, pandas as pd
from keys import google_timezone_api_key
from dateutil import parser as date_parser

In [2]:
# define pause interval to not hammer their server
pause = 0.1

In [3]:
# load the data, parse datetime string to datetime object, and combine lat-long into single column
df = pd.read_csv('data/untappd_details_geocoded.csv', encoding='utf-8')
df['date_pacific_tz'] = df['date_pacific_tz'].map(lambda x: date_parser.parse(x))
df['venue_latlng'] = df.apply(lambda row: '{},{}'.format(row['venue_lat'], row['venue_lon']), axis=1)
df.head()

,date_pacific_tz,beer_name,beer_style,brewery_name,brewery_place,brewery_type,rating,beer_avg_rating,brewery_avg_rating,beer_abv,...,venue_lat,venue_lon,venue_total_checkins,checkin_id,beer_url,brewery_url,venue_url,brewery_lat,brewery_lon,venue_latlng
0,2016-08-08 12:44:47-07:00,51st State Pale Ale,Pale Ale - American,Klamath Basin Brewing,"Klamath Falls, OR United States",Micro Brewery,4.00,3.46,3.56,5.20,...,42.2280,-121.777,1660.0,346221201,https://untappd.com/b/klamath-basin-brewing-51...,https://untappd.com/w/klamath-basin-brewing/2600,https://untappd.com/v/klamath-basin-brewing-co...,42.224867,-121.781670,"42.228,-121.777"
1,2016-08-07 19:05:36-07:00,Depave IPA,IPA - American,McMenamins,"Portland, OR United States",Micro Brewery,4.00,3.60,3.60,7.22,...,45.5646,-122.630,5065.0,346107123,https://untappd.com/b/mcmenamins-depave-ipa/11...,https://untappd.com/w/mcmenamins/3674,https://untappd.com/v/mcmenamins-kennedy-schoo...,45.523062,-122.676482,"45.5646,-122.63"
2,2016-08-06 22:51:04-07:00,Grevensteiner Landbier,Kellerbier / Zwickelbier,Brauerei Veltins,"59872 Meschede, NW Germany",Micro Brewery,3.00,3.46,3.16,5.20,...,45.5258,-122.637,3757.0,345728322,https://untappd.com/b/brauerei-veltins-grevens...,https://untappd.com/w/brauerei-veltins/207,https://untappd.com/v/stammtisch/1596315,51.329257,8.257541,"45.5258,-122.637"
3,2016-08-06 20:22:14-07:00,503 IPA,IPA - American,Migration Brewing,"Portland, OR United States",Micro Brewery,4.25,3.85,3.65,7.30,...,45.5263,-122.636,9025.0,345683595,https://untappd.com/b/migration-brewing-503-ip...,https://untappd.com/w/migration-brewing/4838,https://untappd.com/v/migration-brewing/10357,45.523062,-122.676482,"45.5263,-122.636"
4,2016-08-06 20:21:53-07:00,ISA Got Back,IPA - Session / India Session Ale,Migration Brewing,"Portland, OR United States",Micro Brewery,3.75,3.76,3.65,4.40,...,45.5263,-122.636,9025.0,345683439,https://untappd.com/b/migration-brewing-isa-go...,https://untappd.com/w/migration-brewing/4838,https://untappd.com/v/migration-brewing/10357,45.523062,-122.676482,"45.5263,-122.636"


In [4]:
# how many total venue lat-longs are there, and how many unique lat-longs are there?
print(len(df['venue_latlng']))

venue_latlngs_unique = pd.Series(df['venue_latlng'].unique())
print(len(venue_latlngs_unique))

1540
451


In [5]:
venue_latlngs_unique = venue_latlngs_unique.sort_values()

In [6]:
# send each unique lat-long to the google timezone api to retrieve the local time zone id at that location
def get_timezone_google(latlng, timestamp=0):
    time.sleep(pause)
    url = 'https://maps.googleapis.com/maps/api/timezone/json?location={}&timestamp={}&key={}'
    request = url.format(latlng, timestamp, google_timezone_api_key)
    response = requests.get(request)
    data = response.json()
    try:
        return data['timeZoneId']
    except:
        return None
    
timezones = venue_latlngs_unique.map(get_timezone_google)

In [7]:
# create a dict with key of lat-long and value of timezone
latlng_timezone = {}
for label in timezones.index:
    key = venue_latlngs_unique[label]
    val = timezones[label]
    latlng_timezone[key] = val

In [8]:
# for each row in the df, look up the lat-long in the dict to get the local timezone
def get_timezone_from_dict(venue_latlng):
    try:
        return latlng_timezone[venue_latlng]
    except:
        return None

df['venue_timezone'] = df['venue_latlng'].map(get_timezone_from_dict)
df = df.drop('venue_latlng', axis=1)

In [9]:
# backfill timezones from the next earlier observation as this is more likely to be accurate...
# ...than randomly using the default timezone
df['venue_timezone'] = df['venue_timezone'].fillna(method='bfill')

In [10]:
# convert each row's datetime to the local timezone of the venue i checked into
def localize_date_time(row):
    date_time = row['date_pacific_tz']
    local_timezone = row['venue_timezone']
    try:
        return date_time.astimezone(pytz.timezone(local_timezone))
    except:
        return None
    
df['date_local_tz'] = df.apply(localize_date_time, axis=1)

In [11]:
# look at the first 10 venues and their timezones
df[['venue_name', 'venue_place', 'venue_timezone', 'date_pacific_tz', 'date_local_tz']].head(10)

,venue_name,venue_place,venue_timezone,date_pacific_tz,date_local_tz
0,Klamath Basin Brewing Co.,"1320 Main St Klamath Falls, OR",America/Los_Angeles,2016-08-08 12:44:47-07:00,2016-08-08 12:44:47-07:00
1,McMenamins Kennedy School,"5736 NE 33rd Ave Portland, OR",America/Los_Angeles,2016-08-07 19:05:36-07:00,2016-08-07 19:05:36-07:00
2,Stammtisch,"401 NE 28th Ave Portland, OR",America/Los_Angeles,2016-08-06 22:51:04-07:00,2016-08-06 22:51:04-07:00
3,Migration Brewing,"2828 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 20:22:14-07:00,2016-08-06 20:22:14-07:00
4,Migration Brewing,"2828 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 20:21:53-07:00,2016-08-06 20:21:53-07:00
5,Migration Brewing,"2828 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 20:09:18-07:00,2016-08-06 20:09:18-07:00
6,Migration Brewing,"2828 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 19:53:04-07:00,2016-08-06 19:53:04-07:00
7,Migration Brewing,"2828 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 19:47:18-07:00,2016-08-06 19:47:18-07:00
8,Migration Brewing,"2828 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 19:40:46-07:00,2016-08-06 19:40:46-07:00
9,Slowburger,"2329 NE Glisan St Portland, OR",America/Los_Angeles,2016-08-06 19:02:06-07:00,2016-08-06 19:02:06-07:00


In [12]:
# save to csv
df.to_csv('data/untappd_details_geocoded_timezone.csv', index=False, encoding='utf-8')